In [3]:
import numpy as np
import pandas as pd
from script.data_pipeline import shinyDataFetcher
import plotly.graph_objects as go
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller
from statsmodels.regression.rolling import RollingOLS
import matplotlib.pyplot as plt
import itertools
import warnings

warnings.filterwarnings("ignore")

EV_stocks = ["TSLA", "RIVN", "LCID", "F", "GM", "NIO", "XPEV", "BYDDF", "LI"]
EV_stocks_data = {}
for stock in EV_stocks:
    EV_stocks_data[stock] = shinyDataFetcher(asset=stock, durationStr="3 Y", barSizeSetting="1 day").fetch_asset_data()

log_prices = {}
for ticker in EV_stocks:
    df_temp = EV_stocks_data[ticker].copy()
    df_temp["log_close"] = np.log(df_temp["close"])
    log_prices[ticker] = df_temp.set_index("timestamp")["log_close"].dropna()

# Loop over all unique pairs
for s1, s2 in itertools.combinations(EV_stocks, 2):
    # Align datasets by timestamp
    print(f"Processing pair: {s1} & {s2}")
    df_pair = pd.concat([log_prices[s1], log_prices[s2]], axis=1).dropna()
    if df_pair.empty:
        continue

    X = log_prices[s1]
    Y = log_prices[s2]
    X_const = sm.add_constant(X)  # Adds a constant term to the predictor
    model = sm.OLS(Y, X_const).fit()
    print(f"OLS Coefficients: {model.params.to_dict()}, OLS p-value: {model.pvalues[1]:.4f}, OLS R-squared: {model.rsquared:.4f}")

    # Perform cointegration test
    result = coint(df_pair.iloc[:, 0], df_pair.iloc[:, 1])
    test_stat, p_value, crit_values = result

    print(f"Test Statistic: {test_stat:.3f}, p-value: {p_value:.4f}", f"Critical Values: {crit_values}")
    print(f"r-square > 0.5? {'Yes' if model.rsquared > 0.5 else 'No'}")
    print(f"Is cointegrated? {'Yes' if p_value < 0.05 else 'No'}")
    print("-" * 50)

Processing pair: TSLA & RIVN
OLS Coefficients: {'const': 1.2918768117947566, 'log_close': 0.28832167130623293}, OLS p-value: 0.0000, OLS R-squared: 0.0341
Test Statistic: -2.112, p-value: 0.4702 Critical Values: [-3.91108253 -3.34427805 -3.05010223]
r-square > 0.5? No
Is cointegrated? No
--------------------------------------------------
Processing pair: TSLA & LCID
OLS Coefficients: {'const': 2.1031951858154256, 'log_close': -0.06981964193177112}, OLS p-value: 0.4775, OLS R-squared: 0.0007
Test Statistic: -2.630, p-value: 0.2257 Critical Values: [-3.91108253 -3.34427805 -3.05010223]
r-square > 0.5? No
Is cointegrated? No
--------------------------------------------------
Processing pair: TSLA & F
OLS Coefficients: {'const': 2.8975385680312797, 'log_close': -0.07544623023334526}, OLS p-value: 0.0000, OLS R-squared: 0.0238
Test Statistic: -3.016, p-value: 0.1063 Critical Values: [-3.91108253 -3.34427805 -3.05010223]
r-square > 0.5? No
Is cointegrated? No
--------------------------------

In [10]:
fig = go.Figure()
asset_a = "TSLA"
asset_b = "RIVN"
fig.add_trace(go.Scatter(x=EV_stocks_data[asset_a]["timestamp"], y=np.log(EV_stocks_data[asset_a]["close"]), mode="lines", name=f"{asset_a}"))
fig.add_trace(go.Scatter(x=EV_stocks_data[asset_b]["timestamp"], y=np.log(EV_stocks_data[asset_b]["close"]), mode="lines", name=f"{asset_b}"))
fig.update_layout(title=f"{asset_a}  vs. {asset_b} Stock Log Prices", xaxis_title="timestamp", yaxis_title="Price (USD)", legend=dict(x=0, y=1))
fig.show()
fig.write_html("html_plot/log_price.html")

In [5]:
X = np.log(EV_stocks_data[asset_a]["close"])
Y = np.log(EV_stocks_data[asset_b]["close"])
X_const = sm.add_constant(X)  # Adds a constant term to the predictor
model = sm.OLS(Y, X_const).fit()
print(model.summary())
import statsmodels.tsa.stattools as ts

result = ts.coint(X, Y)
print("Cointegration test result:")
print("========================================")
print("Test Statistic:", result[0])
print("p-value:", result[1])

                            OLS Regression Results                            
Dep. Variable:                  close   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     26.45
Date:                Thu, 10 Apr 2025   Prob (F-statistic):           3.45e-07
Time:                        16:25:56   Log-Likelihood:                -375.63
No. Observations:                 752   AIC:                             755.3
Df Residuals:                     750   BIC:                             764.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2919      0.305      4.234      0.0

In [6]:
def rolling_adf(spread_series, time_index, window=90):
    adf_stats = []
    p_values = []
    dates = []

    for i in range(window, len(spread_series)):
        window_data = spread_series.iloc[i - window : i]
        result = adfuller(window_data, maxlag=1, autolag=None)
        adf_stats.append(result[0])  # Test statistic
        p_values.append(result[1])  # p-value
        dates.append(time_index.iloc[i])
    return pd.DataFrame({"timestamp": dates, "adf_stat": adf_stats, "p_value": p_values})


trade_df = pd.merge(EV_stocks_data["RIVN"][["timestamp", "close"]], EV_stocks_data["TSLA"][["timestamp", "close"]], on="timestamp", suffixes=("_RIVN", "_TSLA"))
trade_df["log_close_TSLA"] = np.log(trade_df["close_TSLA"])
trade_df["log_close_RIVN"] = np.log(trade_df["close_RIVN"])
X = trade_df["log_close_TSLA"].astype(float)
Y = trade_df["log_close_RIVN"].astype(float)
X_with_const = sm.add_constant(X)
model = RollingOLS(endog=Y, exog=X_with_const, window=20)
rres = model.fit()
trade_df["hedge_ratio"] = rres.params["log_close_TSLA"]
trade_df["spread"] = trade_df["log_close_RIVN"] - trade_df["hedge_ratio"] * trade_df["log_close_TSLA"]
trade_df["z_score"] = (trade_df["spread"] - trade_df["spread"].rolling(window=20).mean()) / trade_df["spread"].rolling(window=20).std()
trade_df.dropna(inplace=True)
rolling_adf_df = rolling_adf(trade_df["spread"], trade_df["timestamp"], window=60)
trade_df = pd.merge(trade_df, rolling_adf_df, on="timestamp", how="left")
trade_df.dropna(inplace=True)

In [11]:
fig = go.Figure()
rolling_window = 60
fig.add_trace(go.Scatter(x=rolling_adf_df["timestamp"], y=rolling_adf_df["p_value"], mode="lines", name="Rolling ADF p-value"))
fig.add_hline(y=0.05, line_dash="dash", line_color="red", annotation_text="Significance threshold (0.05)", annotation_position="bottom right")
fig.update_layout(title=f"Rolling ADF Test on Spread (window={rolling_window})", xaxis_title="timestamp", yaxis_title="p-value")
fig.show()
fig.write_html('html_plot/adf_test.html')

In [12]:
# Step-by-step trade logic
df = trade_df.copy()
df["position"] = 0  # 1: long spread, -1: short spread
df["trade_entry"] = False
df["trade_exit"] = False
df["pnl"] = 0.0

in_position = False
position_type = 0
entry_index = None
entry_spread = None

for i in range(len(df)):
    z = df["z_score"].iloc[i]
    spread = df["spread"].iloc[i]

    if not in_position:
        if z > 2:
            df.at[df.index[i], "position"] = -1
            df.at[df.index[i], "trade_entry"] = True
            in_position = True
            position_type = -1
            entry_index = i
            entry_spread = spread
        elif z < -2:
            df.at[df.index[i], "position"] = 1
            df.at[df.index[i], "trade_entry"] = True
            in_position = True
            position_type = 1
            entry_index = i
            entry_spread = spread
    else:
        stop_loss_triggered = (z > 3 and position_type == -1) or (z < -3 and position_type == 1)
        mean_reversion_triggered = abs(z) < 0.5
        timeout_triggered = i - entry_index >= 10

        if stop_loss_triggered or mean_reversion_triggered or timeout_triggered:
            df.at[df.index[i], "position"] = 0
            df.at[df.index[i], "trade_exit"] = True
            pnl = -(spread - entry_spread)
            df.at[df.index[i], "pnl"] = pnl
            in_position = False
            position_type = 0
            entry_index = None
            entry_spread = None
        else:
            df.at[df.index[i], "position"] = position_type

# Assuming 'df' is your DataFrame from the previous cell containing the computed spread, trade_entry, and trade_exit columns
spread_mean = df["spread"].mean()
spread_std = df["spread"].std()
fig = go.Figure()
# Plot the spread line
fig.add_trace(go.Scatter(x=df.index, y=df["spread"], mode="lines", name="Spread"))
# Plot horizontal lines: zero, ±2σ, and ±0.5σ
fig.add_trace(go.Scatter(x=[df.index[0], df.index[-1]], y=[0, 0], mode="lines", name="Zero Line", line=dict(dash="dash", color="black")))
fig.add_trace(go.Scatter(x=[df.index[0], df.index[-1]], y=[spread_mean + 2 * spread_std, spread_mean + 2 * spread_std], mode="lines", name="+2σ", line=dict(dash="dash", color="red")))
fig.add_trace(go.Scatter(x=[df.index[0], df.index[-1]], y=[spread_mean - 2 * spread_std, spread_mean - 2 * spread_std], mode="lines", name="-2σ", line=dict(dash="dash", color="red")))
fig.add_trace(go.Scatter(x=[df.index[0], df.index[-1]], y=[spread_mean + 0.5 * spread_std, spread_mean + 0.5 * spread_std], mode="lines", name="+0.5σ", line=dict(dash="dash", color="green")))
fig.add_trace(go.Scatter(x=[df.index[0], df.index[-1]], y=[spread_mean - 0.5 * spread_std, spread_mean - 0.5 * spread_std], mode="lines", name="-0.5σ", line=dict(dash="dash", color="green")))
# Scatter markers for trade entries and exits
entries_df = df[df["trade_entry"]]
exits_df = df[df["trade_exit"]]
fig.add_trace(go.Scatter(x=entries_df.index, y=entries_df["spread"], mode="markers", name="Entry", marker=dict(color="blue", size=10)))
fig.add_trace(go.Scatter(x=exits_df.index, y=exits_df["spread"], mode="markers", name="Exit", marker=dict(color="purple", size=10)))
fig.update_layout(title="Spread and Trade Entry/Exit Points", xaxis_title="Timestamp", yaxis_title="Spread", template="plotly_white")
fig.show()
fig.write_html('html_plot/spread.html')

In [13]:
# Create cumulative PnL column
df["cumulative_pnl"] = df["pnl"].cumsum()

# Trade summary metrics
total_trades = df["trade_exit"].sum()
total_pnl = df["pnl"].sum()
winning_trades = df[df["pnl"] > 0]["pnl"].count()
losing_trades = df[df["pnl"] < 0]["pnl"].count()
avg_return = df["pnl"][df["pnl"] != 0].mean()
win_rate = winning_trades / total_trades if total_trades > 0 else np.nan
entry_dates = df.loc[df["trade_entry"], "timestamp"].reset_index(drop=True)
exit_dates = df.loc[df["trade_exit"], "timestamp"].reset_index(drop=True)
holding_durations = [(exit_date - entry_date).days for entry_date, exit_date in zip(entry_dates, exit_dates)]
avg_holding_period = np.mean(holding_durations) if holding_durations else np.nan


# Summary dictionary
summary = {
    "Total Trades": total_trades,
    "Total PnL": round(total_pnl, 4),
    "Winning Trades": winning_trades,
    "Losing Trades": losing_trades,
    "Win Rate (%)": round(win_rate * 100, 2) if not np.isnan(win_rate) else "N/A",
    "Average Trade Return": round(avg_return, 4) if not np.isnan(avg_return) else "N/A",
    "Average Holding Period (days)": round(avg_holding_period, 2) if not np.isnan(avg_holding_period) else "N/A",
}

# Plot cumulative PnL with Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df["cumulative_pnl"], mode="lines", name="Cumulative PnL"))
fig.update_layout(
    title="Cumulative PnL Over Time",
    xaxis_title="Date",
    yaxis_title="Cumulative PnL"
)
fig.show()
fig.write_html("html_plot/cumulative_pnl.html")

In [ ]:
import pprint
pprint.pprint(summary)

{'Average Holding Period (days)': np.float64(9.26),
 'Average Trade Return': np.float64(0.5212),
 'Losing Trades': np.int64(17),
 'Total PnL': np.float64(19.8054),
 'Total Trades': np.int64(38),
 'Win Rate (%)': np.float64(55.26),
 'Winning Trades': np.int64(21)}


In [15]:
trade_df.to_html('blotter.html')

In [21]:
from IPython.display import HTML, display

import itables
html = itables.to_html_datatable(trade_df, display_logo_when_loading=False)
with open('../docs/dataframe/blotter.html', 'w') as f:
    f.write(html)
html

'<table id="itables_47f25ca1_8518_4a0d_b6d4_0348df35d875" class="display nowrap" data-quarto-disable-processing="true" style="table-layout:auto;width:auto;margin:auto;caption-side:bottom">\n<thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>timestamp</th>\n      <th>close_RIVN</th>\n      <th>close_TSLA</th>\n      <th>log_close_TSLA</th>\n      <th>log_close_RIVN</th>\n      <th>hedge_ratio</th>\n      <th>spread</th>\n      <th>z_score</th>\n      <th>adf_stat</th>\n      <th>p_value</th>\n    </tr>\n  </thead><tbody><tr>\n<td style="vertical-align:middle; text-align:left">\n\nLoading ITables v2.3.0 from the internet...\n(need <a href=https://mwouts.github.io/itables/troubleshooting.html>help</a>?)</td>\n</tr></tbody>\n</table>\n<link href="https://www.unpkg.com/dt_for_itables@2.2.0/dt_bundle.css" rel="stylesheet">\n<script type="module">\n    import {DataTable, jQuery as $} from \'https://www.unpkg.com/dt_for_itables@2.2.0/dt_bundle.js\';\n\n    document.queryS

In [19]:
display(HTML(html))